In [1]:
from model_selection import *
from preprocessing import *

In [2]:
X_train = load().pipe(extract_features).pipe(drop_duplicates, drop_long_text=True).pipe(clean_text).pipe(text_mining_sentiment).pipe(add_user_text)
X_test = load().pipe(extract_features).pipe(clean_text).pipe(text_mining_sentiment).pipe(add_user_text)


X_train, X_test = text_mining_tfdf(X_train, X_test, min_df=0.01) #if you don't want to use it, just comment it out

X_train, X_test = add_word_embeddings(X_train, X_test) #if you don't want to use it, just comment it out

X_train = X_train.pipe(convert_categorical) #necessary
X_test = X_test.pipe(convert_categorical)   #necessary

#cols_to_drop = ["day_of_week", "month_of_year", "day_of_month", "hour_of_day"]

#X_train.drop(columns= cols_to_drop, inplace= True)
#X_test.drop(columns= cols_to_drop, inplace= True)


print('Start Normalizing')
X_train, X_test, y_train = normalize(X_train, X_test)
print('Finish Normalizing')

100%|██████████| 74999/74999 [00:14<00:00, 5290.83it/s]
/home/christian/.local/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/christian/.local/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'doe', 'ha', 'might', 'must', "n't", 'need', 'sha', 'wa', 'wo', 'would'] not in stop_words.
  warnings.warn(
/home/christian/.local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Sto facendo autovalidation


Progress: 100.0% Trials:    9 Best score:  1.000000 ETA:   0h 0m 0s
Training again with best arguments
Read 2M words
Number of words:  248188
Number of labels: 2
Progress: 100.0% words/sec/thread:  896547 lr:  0.000000 avg.loss:  0.014246 ETA:   0h 0m 0s


Start Normalizing
Finish Normalizing


In [3]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler 


ros = RandomOverSampler(random_state=42)

X_train, y_train = ros.fit_resample(X_train, y_train)
print(f'Resampled dataset shape {Counter(y_train)}' )


Resampled dataset shape Counter({1: 127558, 0: 127558})


In [4]:
clf = RandomForestClassifier(n_estimators = 1200, min_samples_split = 14, min_samples_leaf =  14, max_features = 'sqrt', criterion =  'entropy', bootstrap= True,random_state=42, n_jobs=-1)
print('Start Training')
clf.fit(X_train,y_train)
print('Finish Training')

y_pred = clf.predict(X_test)

filepath = "prova11.csv"
save_results(y_pred, fp=filepath)
print("File Salvato!")

Start Training
Finish Training
File Salvato!


In [5]:
X_train

,ids,day_of_week,month_of_year,day_of_month,hour_of_day,char_count,neg,neu,pos,compound,...,week,well,wish,work,would,x,yeah,yes,embedding_negativity,embedding_positivity
0,0.425080,0.000000,0.5,0.566667,0.043478,0.992537,0.092,0.625,0.283,0.780425,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.277123,0.000061,9.999391e-01
1,0.594974,1.000000,0.5,1.000000,0.260870,0.611940,0.000,0.768,0.232,0.804351,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000106,9.998943e-01
2,0.611332,0.000000,1.0,0.000000,0.478261,0.253731,0.000,1.000,0.000,0.499621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000001,9.999990e-01
3,0.414373,1.000000,0.5,0.533333,0.086957,0.343284,0.000,0.722,0.278,0.702489,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,6.663504e-08
4,0.619215,0.166667,1.0,0.033333,0.000000,0.462687,0.000,0.874,0.126,0.538590,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,3.330906e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255111,0.874414,0.500000,1.0,0.566667,0.173913,0.761194,0.082,0.773,0.145,0.660239,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.999982,1.841960e-05
255112,0.630110,0.166667,1.0,0.033333,0.739130,0.014925,0.000,1.000,0.000,0.499621,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,6.866685e-10
255113,0.686906,0.833333,1.0,0.166667,0.739130,0.746269,0.266,0.583,0.151,0.236333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.999998,2.379717e-06
255114,0.634784,0.333333,1.0,0.066667,0.086957,0.156716,0.000,0.488,0.512,0.821109,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.705330,2.946700e-01


In [6]:
clf = LinearSVC()
print('Start Training')
clf.fit(X_train,y_train)
print('Finish Training')

y_pred = clf.predict(X_test)

filepath = "8 textmining005_user_added.csv"
save_results(y_pred, fp=filepath)
print("File Salvato!")

Start Training
Finish Training
File Salvato!


In [8]:
importances = pd.Series(X_train.columns, clf.feature_importances_)
importances.sort_index(ascending = False).head(20)

0.436828    embedding_positivity
0.417560    embedding_negativity
0.072519                     ids
0.023969                compound
0.016053                     neg
0.010298                     pos
0.007789              char_count
0.006131                polarity
0.004997                     neu
0.003855            subjectivity
dtype: object

In [9]:
clf = LinearSVC()
print('Start Training')
clf.fit(X_train,y_train)
print('Finish Training')

y_pred = clf.predict(X_test)

filepath = "4_notextmining_LSVC.csv"
save_results(y_pred, fp=filepath)
print("File Salvato!")

Start Training
Finish Training
File Salvato!


In [1]:
X_train

NameError: name 'X_train' is not defined

In [35]:
tweets = load('development.csv')
y = tweets.pop('sentiment')
X = tweets

X_train_hp, X_test_hp, y_train_hp, y_test_hp = train_test_split(X, y, train_size= 0.8, test_size=0.2, random_state= 42)
X_train_hp = pd.concat([X_train_hp, y_train_hp], axis=1)

X_train = X_train_hp.pipe(extract_features).pipe(drop_duplicates, drop_long_text=True).pipe(clean_text).pipe(text_mining_sentiment).pipe(add_user_text)
X_test = X_test_hp.pipe(extract_features).pipe(clean_text).pipe(text_mining_sentiment).pipe(add_user_text)

#X_train, X_test = text_mining_tfdf(X_train, X_test) #if you don't want to use it, just comment it out

X_train, X_test = add_word_embeddings(X_train, X_test) #if you don't want to use it, just comment it out

X_train = X_train.pipe(convert_categorical) #necessary
X_test = X_test.pipe(convert_categorical)   #necessary

print('Start Normalizing')
X_train, X_test, y_train = normalize(X_train, X_test)
print('Finish Normalizing')


/home/christian/Desktop/DLS-Project/DSL2122_january_dataset/preprocessing.py:135: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets['text'] = tweets['text'].str.replace(pat= pat, repl = repl, regex=True)
100%|██████████| 178472/178472 [11:14<00:00, 264.42it/s]
/home/christian/Desktop/DLS-Project/DSL2122_january_dataset/preprocessing.py:184: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets["neg"] = sentiment[:, 0]
/home/christian/Desktop/DLS-Project/DSL2122_january_dataset/preprocessing.py:185: Settin

Sto facendo autovalidation


Progress: 100.0% Trials:   12 Best score:  0.999993 ETA:   0h 0m 0s
Training again with best arguments
Read 2M words
Number of words:  215890
Number of labels: 2
Progress: 100.0% words/sec/thread: 1242399 lr:  0.000000 avg.loss:  0.013965 ETA:   0h 0m 0s


Start Normalizing
Finish Normalizing


In [39]:
# Use the random grid to search for best hyperparameters
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestClassifier()

parameters_random_forest = {
    "n_estimators" : np.arange(start= 300, stop = 1500, step = 300), 
    "max_features" : ["sqrt", "log2", None, 0.3, 0.4, 0.5, 0.6], 
    #"max_depth ": np.arange(start= 15, stop = 30, step = 5), 
    "criterion": ["gini", "entropy"],
    "min_samples_split" :np.arange(start= 2, stop = 20, step = 3),
    "min_samples_leaf":np.arange(start= 2, stop = 20, step = 3),
    "bootstrap" : [True, False]
}

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = parameters_random_forest, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1, scoring= 'f1_macro')
# Fit the random search model
rf_random.fit(X_train, y_train)

print(rf_random.best_params_)
print()
print(rf_random.best_score_)
print()
print(rf_random.best_estimator_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] END bootstrap=True, criterion=entropy, max_features=0.6, min_samples_leaf=14, min_samples_split=17, n_estimators=300; total time= 3.9min
[CV] END bootstrap=True, criterion=entropy, max_features=0.6, min_samples_leaf=14, min_samples_split=17, n_estimators=300; total time= 4.1min
[CV] END bootstrap=True, criterion=entropy, max_features=0.6, min_samples_leaf=14, min_samples_split=17, n_estimators=300; total time= 4.1min
[CV] END bootstrap=True, criterion=entropy, max_features=sqrt, min_samples_leaf=17, min_samples_split=8, n_estimators=900; total time= 5.2min
[CV] END bootstrap=True, criterion=entropy, max_features=sqrt, min_samples_leaf=17, min_samples_split=8, n_estimators=900; total time= 5.3min
[CV] END bootstrap=True, criterion=entropy, max_features=sqrt, min_samples_leaf=17, min_samples_split=8, n_estimators=900; total time= 5.3min
[CV] END bootstrap=True, criterion=gini, max_features=0.4, min_samples_leaf=11, min_s

/home/christian/.local/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END bootstrap=True, criterion=entropy, max_features=0.5, min_samples_leaf=8, min_samples_split=8, n_estimators=900; total time=11.5min
[CV] END bootstrap=False, criterion=gini, max_features=0.6, min_samples_leaf=2, min_samples_split=2, n_estimators=1200; total time=29.1min
[CV] END bootstrap=True, criterion=entropy, max_features=0.5, min_samples_leaf=8, min_samples_split=8, n_estimators=900; total time=11.6min
[CV] END bootstrap=False, criterion=entropy, max_features=0.4, min_samples_leaf=11, min_samples_split=17, n_estimators=600; total time= 8.3min
[CV] END bootstrap=False, criterion=entropy, max_features=0.4, min_samples_leaf=11, min_samples_split=17, n_estimators=600; total time= 8.7min
[CV] END bootstrap=False, criterion=entropy, max_features=log2, min_samples_leaf=11, min_samples_split=8, n_estimators=900; total time= 7.9min
[CV] END bootstrap=False, criterion=entropy, max_features=log2, min_samples_leaf=11, min_samples_split=8, n_estimators=900; total time= 8.0min
[CV] END 

In [40]:
print(rf_random.best_params_)
print()
print(rf_random.best_score_)
print()
print(rf_random.best_estimator_)

{'n_estimators': 1200, 'min_samples_split': 14, 'min_samples_leaf': 14, 'max_features': 'sqrt', 'criterion': 'entropy', 'bootstrap': True}

0.9674699361940174

RandomForestClassifier(criterion='entropy', max_features='sqrt',
                       min_samples_leaf=14, min_samples_split=14,
                       n_estimators=1200)


In [12]:
X_train

,ids,day_of_week,month_of_year,day_of_month,hour_of_day,char_count,neg,neu,pos,compound,...,x,xx,ya,yay,yeah,year,yes,yet,embedding_negativity,embedding_positivity
0,0.425080,0.000000,0.5,0.566667,0.043478,0.930070,0.092,0.625,0.283,0.781884,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.249338,0.0,1.189352e-05,9.999881e-01
1,0.594974,1.000000,0.5,1.000000,0.260870,0.573427,0.000,0.768,0.232,0.805978,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,5.293700e-04,9.994707e-01
2,0.611332,0.000000,1.0,0.000000,0.478261,0.237762,0.000,1.000,0.000,0.499110,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,2.544284e-07,9.999998e-01
3,0.414373,1.000000,0.5,0.533333,0.086957,0.321678,0.000,0.722,0.278,0.703401,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,9.999998e-01,2.859269e-07
4,0.619215,0.166667,1.0,0.033333,0.000000,0.433566,0.000,0.843,0.157,0.538352,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,9.999971e-01,2.875988e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222600,0.921197,0.833333,1.0,0.633333,0.869565,0.496503,0.000,0.563,0.437,0.822854,...,0.0,0.0,0.0,0.0,0.394405,0.0,0.000000,0.0,9.990305e-01,9.694987e-04
222601,0.605527,0.000000,1.0,0.000000,0.043478,0.132867,0.000,1.000,0.000,0.499110,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,4.391322e-07,9.999995e-01
222602,0.607632,0.000000,1.0,0.000000,0.260870,0.468531,0.227,0.625,0.148,0.326285,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,9.999919e-01,8.100567e-06
222603,0.896096,0.666667,1.0,0.600000,0.347826,0.321678,0.612,0.388,0.000,0.063336,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,1.000000e+00,3.133846e-08


In [1]:

from model_selection import *
from preprocessing import *
from tuning import *

def model_selection(use_text_tf_df: bool, use_wor_embeddings: bool):

    dataframe = load()
    y = dataframe.pop('sentiment')
    X = dataframe

    print("Separo il dataset in training e test set")
    X_train_hp, X_test_hp, y_train_hp, y_test = train_test_split(X, y, train_size= 0.8, test_size=0.2, random_state= 42)
    
    X_train_hp = X_train_hp.sample(n=10000, random_state=42)
    X_test_hp = X_test_hp.sample(n=10000, random_state=42)
    y_train_hp = y_train_hp.sample(n=10000, random_state=42)
    y_test = y_test.sample(n=10000, random_state=42)
    
    X_train_hp = pd.concat([X_train_hp, y_train_hp], axis=1)

    print("Inizio il preprocessing.")
    X_train = X_train_hp.pipe(extract_features).pipe(drop_duplicates, drop_long_text=True).pipe(clean_text, deep_clean = True)
    X_test = X_test_hp.pipe(extract_features).pipe(clean_text,  deep_clean = True)
    #y_test = y_test.sample(n=10000, random_state=42)

    X_test.drop(columns=["user"], inplace=True)
    X_train.drop(columns=["user"], inplace=True)
    
    if use_text_tf_df:
        print("Uso il text mining")
        X_train, X_test = text_mining_tfdf(X_train, X_test, min_df=0.01) #if you don't want to use it, just comment it out

    if use_wor_embeddings:
        print("Uso il word embeddings")
        X_train, X_test = add_word_embeddings(X_train, X_test) #if you don't want to use it, just comment it out

    X_train = X_train.pipe(convert_categorical) #necessary
    X_test = X_test.pipe(convert_categorical)   #necessary

    print('Start Normalizing')
    X_train_f, X_test_f, y_train_f = normalize(X_train, X_test)
    print('Finish Normalizing')
    
    print("Preprocess finito")

    models = [RandomForestClassifier(), LinearSVC(), HistGradientBoostingClassifier(), BernoulliNB()]
    names = ["Random Forest", "Linear SVC", "Hist Gradient Boosting", "Bernoulli NB"]

    for i,clf in enumerate(models):
        print(f"Start training of {names[i]}")
        start= time.time()
        clf.fit(X_train_f, y_train_f)
        end = time.time()
        print(f'Finish Training with {names[i]} in {end-start} seconds')

        y_pred = clf.predict(X_test_f)
        f1 = f1_score(y_test, y_pred, average= "macro")
        print(f"Precision of {names[i]} is: {f1}\n\n\n\n")

In [ ]:
model_selection(use_text_tf_df=False, use_wor_embeddings=True)

Separo il dataset in training e test set
Inizio il preprocessing.
